In [1]:
# with constant variance = 1
def model(obs_data, mask, sample_means, drug_means):
    nsamples, ndrugs = obs_data.shape
    # define samples
    s = []
    for i in range(nsamples):
        sample_name = 's' + str(i)
        s.append(pyro.sample(sample_name, dist.Normal(sample_means[i], 1)))
    # define drugs
    d = []
    for i in range(ndrugs):
        drug_name = 'd' + str(i)
        d.append(pyro.sample(drug_name, dist.Normal(drug_means[i], 1)))
    result = torch.empty((nsamples, ndrugs), dtype=torch.int32)
    sigma = pyro.param("sigma", lambda: torch.ones(()), constraint=constraints.positive)
    for i in range(nsamples):
        for j in range(ndrugs):
            result[i, j] = pyro.sample(f"obs_{i,j}", 
                                       dist.Normal(s[i] * d[j], sigma), 
                                       obs=obs_data[i,j] if mask[i,j] else None)
    return result